In [2]:
import torch

import evaluate
import datasets

from tqdm.auto import tqdm

from transformers import MarianTokenizer, MarianMTModel

In [3]:
# load the translation model from transformers
# model_name = "Helsinki-NLP/opus-mt-ar-en"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to('cuda')


mname = "marefa-nlp/marefa-mt-en-ar"
tokenizer = MarianTokenizer.from_pretrained(mname)
model = MarianMTModel.from_pretrained(mname)


bleu_scorer = evaluate.load("sacrebleu")

test_dataset = datasets.load_dataset('iwslt2017','iwslt2017-ar-en', split='test')

In [4]:
source_language = 'en'
target_language = 'ar'

In [5]:
source_language = 'en'
target_language = 'ar'

In [6]:
input_texts = [example[source_language] for example in test_dataset['translation']][-10:]
targets = [example[target_language] for example in test_dataset['translation']][-10:]
len(input_texts), len(targets), input_texts[:5], targets[:5]

(10,
 10,
 ["One major consequence of this work  is that maybe all of these decades,  we've had the whole concept of cybernetic revolt  in reverse.",
  "It's not that machines first become intelligent  and then megalomaniacal  and try to take over the world.",
  "It's quite the opposite,  that the urge to take control  of all possible futures  is a more fundamental principle  than that of intelligence,  that general intelligence may in fact emerge  directly from this sort of control-grabbing,  rather than vice versa.",
  'Another important consequence is goal seeking.',
  "I'm often asked, how does the ability to seek goals  follow from this sort of framework?"],
 ['أحد العواقب الكبرى لهذا العمل هو أنه لربما طوال كل هذه العقود، كان لدينا المفهوم العكسي للثورة الآلية.',
  'الأمر ليس في أن الآلات تصبح ذكية في البداية ثم ينتابها جنون العظمة و تحاول السيطرة على العالم.',
  'إنه تماماً العكس، أن النزعة للسيطرة على كل الأزمنة المستقبلية الواردة هي مبدأ أساسي أكثر من مبدأ الذكاء، أن نواحي الذ

In [7]:
model.generation_config.num_beams=1
model.generation_config.use_cache = False
model.generation_config.batch_size=2

In [8]:
def get_batches(inputs,batch_size):
    for i in tqdm(
            range(0, len(inputs), batch_size),
            desc="Generating Sequences",
            total=len(inputs) // batch_size,
        ):
        yield inputs[i : i + batch_size]

In [49]:
def translate(texts):
    translated_texts = list()
    for batch in get_batches(texts,batch_size=model.generation_config.batch_size):
        translated_tokens = model.generate(
            **tokenizer(batch, return_tensors="pt",padding=True),
            # num_beams=4,
            # do_sample=True,
        )
        translated_texts += [tokenizer.decode(t, skip_special_tokens=True) for t in translated_tokens]
    return translated_texts

In [50]:
# Example batch of input sentences
hf_predictions = translate(input_texts)
len(input_texts), len(hf_predictions), len(targets)

Generating Sequences:   0%|          | 0/5 [00:00<?, ?it/s]

(10, 10, 10)

In [51]:
hf_predictions

['ومن النتائج الرئيسية لهذا العمل أنه ربما كانت كل هذه العقود، قد حصل لنا المفهوم الكامل للثورة الإلكترونية بالعكس.',
 'ليس أن الآلات تصبح أولاً ذكية ثم ميگالومانيكال وتحاول الاستيلاء على العالم.',
 'بل العكس تماماً، هو أن الحث على السيطرة على جميع المستقبلات الممكنة هو مبدأ أكثر جوهرية من مبدأ الذكاء، ذلك أن المخابرات العامة قد تخرج في الواقع مباشرة من هذا النوع من التجسس، بدلاً من العكس.',
 'ومن النتائج الهامة الأخرى السعي لتحقيق الهدف.',
 'غالباً ما يُسألني، كيف تتبع القدرة على البحث عن الأهداف من هذا النوع من الإطارات؟',
 'والاجابة هي أن القدرة على السعي لتحقيق الأهداف ستتبع مباشرة من هذا بالمعنى التالي: تماماً كما ستسافر عبر نفق، عنق القنينة في فضاء مسارك المستقبلي، من أجل تحقيق العديد من الأهداف المتنوعة الأخرى لاحقاً، أو مثلك تماماً ستستثمر في أمن مالي، تقلل سيولتك قصيرة المدى من أجل زيادة ثروتك على المدى الطويل، هدف البحث عن البرز مباشرة',
 'من دافع طويل المدى لزيادة حرية العمل في المستقبل.',
 'وأخيراً، كتب ريتشارد ڤاينمان، الفيزيائي الشهير، ذات مرة، أنه لو دمرت الحضارة البشرية

In [52]:
bleu_scorer.compute(predictions=hf_predictions, references=targets)

{'score': 15.796125110909543,
 'counts': [128, 58, 28, 13],
 'totals': [264, 254, 244, 235],
 'precisions': [48.484848484848484,
  22.834645669291337,
  11.475409836065573,
  5.531914893617022],
 'bp': 0.9701515036966302,
 'sys_len': 264,
 'ref_len': 272}

In [11]:
model.generation_config

GenerationConfig {
  "bad_words_ids": [
    [
      62801
    ]
  ],
  "batch_size": 100,
  "bos_token_id": 0,
  "decoder_start_token_id": 62801,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "max_length": 512,
  "pad_token_id": 62801,
  "use_cache": false
}

In [17]:
def generate(inputs, decoder_input_ids):
    model_outputs = model(
        **tokenizer(
            inputs,
            return_tensors="pt",
            padding=True,
        ),
        decoder_input_ids=decoder_input_ids,
    )
    return model_outputs.logits


from generate_sequences import GreedyGenerator

sequences_generator = GreedyGenerator(
    use_tqdm=True,
    device=model.device,
    generate_fn=generate,
    batch_size=model.generation_config.batch_size,
    # max_length=model.generation_config.max_length,
    max_length=50,
    eos_token_id=model.generation_config.eos_token_id,
    decoder_start_token_id=model.generation_config.decoder_start_token_id,
)

In [18]:
prediction_ids = sequences_generator.generate(input_texts)
predictions = tokenizer.batch_decode(prediction_ids, skip_special_tokens=True)
len(input_texts), len(predictions), len(targets)

Generating Sequences:   0%|          | 0/5 [00:00<?, ?it/s]

(10, 10, 10)

In [19]:
predictions

['ومن النتائج الرئيسية لهذا العمل أنه ربما كانت كل هذه العقود، قد حصل لنا المفهوم الكامل للثورة الإلكترونية بالعكس.',
 'ليس أن الآلات تصبح أولاً ذكية ثم ميگالومانيكال وتحاول الاستيلاء على العالم.',
 'بل العكس تماماً، هو أن الحث على السيطرة على جميع المستقبلات الممكنة هو مبدأ أكثر جوهرية من مبدأ الذكاء، ذلك أن المخابرات العامة قد تخرج في الواقع مباشرة من هذا النوع من التجسس، بدلاً من العكس.',
 'ومن النتائج الهامة الأخرى السعي لتحقيق الهدف.',
 'غالباً ما يُسألني، كيف تتبع القدرة على البحث عن الأهداف من هذا النوع من الإطارات؟',
 'والاجابة هي أن القدرة على السعي لتحقيق الأهداف ستتبع مباشرة من هذا بالمعنى التالي: تماماً كما ستسافر عبر نفق، عنق القنينة في فضاء مسارك المستقبلي، من أجل تحقيق العديد من الأهداف المتنوعة الأخرى لاحقاً،',
 'من دافع طويل المدى لزيادة حرية العمل في المستقبل.',
 'وأخيراً، كتب ريتشارد ڤاينمان، الفيزيائي الشهير، ذات مرة، أنه لو دمرت الحضارة البشرية ولم تتمكن من تمرير سوى مفهوم واحد إلى أحفادنا لمساعدتهم على إعادة بناء الحضارة، ينبغي أن يكون ذلك المفهوم أن كل',
 'أما نظ

In [20]:
bleu_scorer.compute(predictions=predictions, references=targets)

{'score': 7.785887799197727,
 'counts': [100, 41, 14, 3],
 'totals': [211, 201, 191, 182],
 'precisions': [47.39336492890995,
  20.398009950248756,
  7.329842931937173,
  1.6483516483516483],
 'bp': 0.7489376628737409,
 'sys_len': 211,
 'ref_len': 272}